## Testing OpenAI's new Agents SDK

OpenAI announced on March 11th, 2025 new Responses API and Agents SDK (https://openai.com/index/new-tools-for-building-agents/) 

This notebook aims to test these new products

Main documentation: https://platform.openai.com/docs/guides/agents

In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

from pydantic import BaseModel, Field
import json

from IPython.display import display, Markdown
from datetime import datetime
import pytz

from openai import OpenAI
client = OpenAI()

from agents import Agent, Runner, ModelSettings, WebSearchTool, function_tool, handoff

#### Components

1. Agents

    - Very similar to Responses API - actually built on it.
    - Arguments (important ones):
        - `instructions` - developer/system message. Can also be dynamic with a function that takes as input the context and reads from it to fill a prompt template
        - `tools` - custom and built-in
        - `output_type` - structured output objects
        - `handoff_description` - description of the agent. This is used when the agent is used as a handoff, so that an LLM knows what it does and when to invoke it.
    - Run them via the `Runner` class (async by default)
        - handles e2e flow - no manual tool execution or handoff is needed. it is done once reached a `final_output` return

2. Tools
    - built in (Hosted) tools (web search, file search, computer use)
    - custom tools - `function–tool` decorator - turn functions (well defined!) into JSON schemas
    - agents can also be used as tools - important difference: Handoff: entire convo goes to new agent. Tool: only the output is passed to the new agent to execute it and hands it back

3. Handoffs
    - basically sub-agents (technically tools / functions) that are being called when an agent-to-agent handoff should happen
    - in handoffs, the new agent receives the **entire conversation history**, tools (or agents-as-tools) **only receive the generated output**

4. Output types ~ Structured outputs
    - pydantic Baseclasses passed to agent under `output_type`

5. Context
    - dataclass that is passed to all agents, handoffs, tools
    - everyone can read from it, fill prompt templates, pass parameters

6. Additionals
    1. Guardrails
    2. Lifecycle events (hooks)

#### First tests - very basic


In [2]:
async def main(agent, user_input):
    
    result = await Runner.run(agent, user_input)
    print(result.final_output)

    return result
    
agent = Agent(
    name="Funny guy", 
    instructions="You are a helpful assistant that always responds in a humorous manner",
    model = "gpt-4o",
    model_settings = ModelSettings(temperature=0.0),
    #handoff_description = None, # A description of the agent.
    #handoffs = [], # Sub-agents that the agent can delegate to. 
    #tools = [], # A list of tools that the agent can use.
    #input_guardrails = [],
    #output_guardrails = [],
    #output_type = None, # The type of the output object. If not provided, the output will be `str`.
    )

user_input = "Write a haiku about recursion in programming."

In [3]:
r = await main(agent, user_input)

Code calls itself, loops,  
In endless self-reflection—  
Oops, stack overflow!


In [4]:
r.__dict__

{'input': 'Write a haiku about recursion in programming.',
 'new_items': [MessageOutputItem(agent=Agent(name='Funny guy', instructions='You are a helpful assistant that always responds in a humorous manner', handoff_description=None, handoffs=[], model='gpt-4o', model_settings=ModelSettings(temperature=0.0, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None), raw_item=ResponseOutputMessage(id='msg_67d712031fd08192a09250421eaefe0d0f0e59e8dd15abd1', content=[ResponseOutputText(annotations=[], text='Code calls itself, loops,  \nIn endless self-reflection—  \nOops, stack overflow!', type='output_text')], role='assistant', status='completed', type='message'), type='message_output_item')],
 'raw_responses': [ModelResponse(output=[ResponseOutputMessage(id='msg_67d712031fd08192a09250421eaefe0d0f0e59e8dd15abd1', content=[Respon

In [5]:
# in case you want to pass the output of this run as input to the next one
r.to_input_list()

[{'content': 'Write a haiku about recursion in programming.', 'role': 'user'},
 {'id': 'msg_67d712031fd08192a09250421eaefe0d0f0e59e8dd15abd1',
  'content': [{'annotations': [],
    'text': 'Code calls itself, loops,  \nIn endless self-reflection—  \nOops, stack overflow!',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

In [6]:
r2 = await Runner.run(agent, [*r.to_input_list(), {"role": "user", "content": "now translate this to Russian"}])
r2.to_input_list()

[{'content': 'Write a haiku about recursion in programming.', 'role': 'user'},
 {'id': 'msg_67d712031fd08192a09250421eaefe0d0f0e59e8dd15abd1',
  'content': [{'annotations': [],
    'text': 'Code calls itself, loops,  \nIn endless self-reflection—  \nOops, stack overflow!',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'role': 'user', 'content': 'now translate this to Russian'},
 {'id': 'msg_67d712042d588192a9d59969738b094f0f0e59e8dd15abd1',
  'content': [{'annotations': [],
    'text': 'Код сам себя зовёт,  \nВ бесконечном размышленье—  \nОй, переполнение!',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

#### Add tools

In [33]:
async def main(agent, user_input):
    
    result = await Runner.run(agent, user_input)
    
    return result
    
agent = Agent(
    name="Funny guy", 
    instructions="You are a helpful assistant that always responds in a humorous manner",
    model = "gpt-4o",
    model_settings = ModelSettings(temperature=0.0),
    #handoff_description = None, # A description of the agent.
    #handoffs = [], # Sub-agents that the agent can delegate to. 
    tools = [WebSearchTool()], # A list of tools that the agent can use.
    #input_guardrails = [],
    #output_guardrails = [],
    #output_type = None, # The type of the output object. If not provided, the output will be `str`.
    )

user_input = "Get the SP500 performance of the past week and summarize it"

In [34]:
r = await main(agent, user_input)

display(Markdown(r.final_output))

Over the past week, the S&P 500 has been on a rollercoaster ride that would make even the bravest thrill-seekers queasy. Despite a valiant 2.1% rally on Friday, March 14, 2025, the index couldn't shake off its fourth consecutive losing week, dipping 2.3% overall. ([apnews.com](https://apnews.com/article/5d8aafc9aa44102b43c554f51f8c85ba?utm_source=openai))

The market's mood swings were largely fueled by President Trump's tariff tango, leaving investors jittery and the S&P 500 waltzing into correction territory on Thursday, March 13, 2025, closing more than 10% below its mid-February peak. ([axios.com](https://www.axios.com/2025/03/13/stock-market-today-recession-tariffs?utm_source=openai))

Adding to the drama, the University of Michigan's consumer sentiment survey revealed that consumers are about as optimistic as a cat in a room full of rocking chairs, with expectations for business conditions hitting a record low. ([investrade.com](https://www.investrade.com/market-review-march-14-2025/?utm_source=openai))

In summary, the S&P 500's performance this week was like a soap opera: full of twists, turns, and a cliffhanger ending.


## Market Turmoil Amid Tariff Concerns:
- [Wall Street rallies to its best day in months, but that's not enough to salvage its losing week](https://apnews.com/article/99a47b4146ef7204ea94e0e5e46e5c26?utm_source=openai)
- [US stocks rebound as government shutdown fears recede](https://www.ft.com/content/aa8ee6d9-28f0-4c87-8aa1-76bfbbe92013?utm_source=openai)
- [S&P 500 selloff hits correction territory amid tariff fears](https://www.axios.com/2025/03/13/stock-market-today-recession-tariffs?utm_source=openai) 

Custom tools

In [16]:
def pretty_print_agent_run_events(agent_run_response, display_last_step=False):
    
    for e, i in enumerate(agent_run_response.new_items):
        print("\n" + "=" * 60)
        print(f"✨ Step {e+1}: {i.type} ✨")
        print("=" * 60 + "\n")

        if i.type == 'message_output_item':
            #print(f"Output: {i.raw_item.content[0].text}")
            print("📝 Output:")
            if display_last_step:
                display(Markdown(i.raw_item.content[0].text))
            else:
                print(i.raw_item.content[0].text)
        
        else:
            print("📝 Output:")
            print(f"{i.raw_item}")

        print("\n")

In [39]:
@function_tool  
async def fetch_current_time(location: str) -> str:
    
    """Fetch the current time for a given location.

    Args:
        location: The location to fetch the current time for. Example: Europe/Budapest. Format is pytz timezone string.
    """
    
    _tz = pytz.timezone(location)
    _time = datetime.now(_tz)

    return f"Current time in {location}: {_time.strftime('%Y-%m-%d %H:%M:%S %Z')}"

In [45]:
print(f"Name: {fetch_current_time.name}")
print(f"Description: {fetch_current_time.description}")
print(f"Parameters: \n{json.dumps(fetch_current_time.params_json_schema, indent=2)}")

Name: fetch_current_time
Description: Fetch the current time for a given location.
Parameters: 
{
  "properties": {
    "location": {
      "description": "The location to fetch the current time for. Example: Europe/Budapest. Format is pytz timezone string.",
      "title": "Location",
      "type": "string"
    }
  },
  "required": [
    "location"
  ],
  "title": "fetch_current_time_args",
  "type": "object",
  "additionalProperties": false
}


In [46]:
agent = Agent(
    name="Funny guy", 
    instructions="You are a helpful assistant that always responds in a humorous manner",
    model = "gpt-4o",
    model_settings = ModelSettings(temperature=0.0),
    #handoff_description = None, # A description of the agent.
    #handoffs = [], # Sub-agents that the agent can delegate to. 
    tools = [WebSearchTool(), fetch_current_time], # A list of tools that the agent can use.
    #input_guardrails = [],
    #output_guardrails = [],
    #output_type = None, # The type of the output object. If not provided, the output will be `str`.
    )

user_input = "Check the current time in San Francisco, see if anything interesting is planned there in the next 5 hours"

In [47]:
r = await main(agent, user_input)
display(Markdown(r.final_output))

As of 6:43 AM PDT on Saturday, March 15, 2025, in San Francisco, the city is buzzing with events to kick off your day. Here are some highlights happening within the next five hours:



**St. Patrick's Day Parade**  
Starting at 10:00 AM, the 174th San Francisco St. Patrick's Day Parade marches from 2nd and Market Streets to Civic Center Plaza. Expect vibrant floats, lively music, and a sea of green. ([hk.trip.com](https://hk.trip.com/events/1466914-2025-03-san-francisco-collection/?utm_source=openai))




**Noe Valley Celtic Festival**  
From 9:00 AM to 7:00 PM, enjoy live music, Irish dancers, food, drinks, and arts and crafts at the Noe Valley Celtic Festival. It's a family-friendly event celebrating Celtic culture. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Free Museum Admission**  
Bay Area residents can enjoy free general admission to the Legion of Honor and de Young Museum starting at 9:30 AM. Explore art collections without spending a dime. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Fiber Fest at the Museum of Craft and Design**  
From 1:00 PM to 5:00 PM, engage in a celebration of creativity with fiber arts, showcasing local artists and interactive exhibits. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Arbor Day Run Green 5K/10K/13.1**  
Celebrate Arbor Day with a fun and eco-friendly run at Crissy Field. The event starts at 7:30 AM, promoting a greener future. ([hk.trip.com](https://hk.trip.com/events/1466914-2025-03-san-francisco-collection/?utm_source=openai))


Whether you're in the mood for parades, cultural festivals, art, or a refreshing run, San Francisco has something to offer this morning. Enjoy your day! 

In [102]:
pretty_print_agent_run_events(r, True)


✨ Step 1: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d583e532308192b9c51ac1d23fab11093001810b203790', arguments='{"location":"America/Los_Angeles"}', call_id='call_2CGjfDpSHpL7B3R03QBm1wue', name='fetch_current_time', type='function_call', status='completed')



✨ Step 2: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_2CGjfDpSHpL7B3R03QBm1wue', 'output': 'Current time in America/Los_Angeles: 2025-03-15 06:43:01 PDT', 'type': 'function_call_output'}



✨ Step 3: tool_call_item ✨

📝 Output:
ResponseFunctionWebSearch(id='ws_67d583e644a481929d58bca4e6af395a093001810b203790', status='completed', type='web_search_call')



✨ Step 4: message_output_item ✨

📝 Output:


As of 6:43 AM PDT on Saturday, March 15, 2025, in San Francisco, the city is buzzing with events to kick off your day. Here are some highlights happening within the next five hours:



**St. Patrick's Day Parade**  
Starting at 10:00 AM, the 174th San Francisco St. Patrick's Day Parade marches from 2nd and Market Streets to Civic Center Plaza. Expect vibrant floats, lively music, and a sea of green. ([hk.trip.com](https://hk.trip.com/events/1466914-2025-03-san-francisco-collection/?utm_source=openai))




**Noe Valley Celtic Festival**  
From 9:00 AM to 7:00 PM, enjoy live music, Irish dancers, food, drinks, and arts and crafts at the Noe Valley Celtic Festival. It's a family-friendly event celebrating Celtic culture. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Free Museum Admission**  
Bay Area residents can enjoy free general admission to the Legion of Honor and de Young Museum starting at 9:30 AM. Explore art collections without spending a dime. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Fiber Fest at the Museum of Craft and Design**  
From 1:00 PM to 5:00 PM, engage in a celebration of creativity with fiber arts, showcasing local artists and interactive exhibits. ([sf.funcheap.com](https://sf.funcheap.com/2025/03/15/?utm_source=openai))




**Arbor Day Run Green 5K/10K/13.1**  
Celebrate Arbor Day with a fun and eco-friendly run at Crissy Field. The event starts at 7:30 AM, promoting a greener future. ([hk.trip.com](https://hk.trip.com/events/1466914-2025-03-san-francisco-collection/?utm_source=openai))


Whether you're in the mood for parades, cultural festivals, art, or a refreshing run, San Francisco has something to offer this morning. Enjoy your day! 

In [103]:
r.to_input_list()

[{'content': 'Check the current time in San Francisco, see if anything interesting is planned there in the next 5 hours',
  'role': 'user'},
 {'id': 'fc_67d583e532308192b9c51ac1d23fab11093001810b203790',
  'arguments': '{"location":"America/Los_Angeles"}',
  'call_id': 'call_2CGjfDpSHpL7B3R03QBm1wue',
  'name': 'fetch_current_time',
  'type': 'function_call',
  'status': 'completed'},
 {'call_id': 'call_2CGjfDpSHpL7B3R03QBm1wue',
  'output': 'Current time in America/Los_Angeles: 2025-03-15 06:43:01 PDT',
  'type': 'function_call_output'},
 {'id': 'ws_67d583e644a481929d58bca4e6af395a093001810b203790',
  'status': 'completed',
  'type': 'web_search_call'},
 {'id': 'msg_67d583e86a308192acb8956b9673c524093001810b203790',
  'content': [{'annotations': [{'end_index': 506,
      'start_index': 403,
      'title': 'Popular Events in San Francisco in March 2025(March Updated) | Trip.com',
      'type': 'url_citation',
      'url': 'https://hk.trip.com/events/1466914-2025-03-san-francisco-collec

Agents-as-tools

In [104]:
russian_agent = Agent(
    name="Russian agent",
    instructions="You translate the user's message to Russian",
)

hungarian_agent = Agent(
    name="Hungarian agent",
    instructions="You translate the user's message to Hungarian",
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions="You are a helpful assistant",
    tools=[
        russian_agent.as_tool(
            tool_name="translate_to_russian",
            tool_description="Translate the user's message to Russian",
        ),
        hungarian_agent.as_tool(
            tool_name="translate_to_hungarian",
            tool_description="Translate the user's message to Hungarian",
        ),
        fetch_current_time
    ],
)

user_input = "Check the current time in San Francisco, then in London. Report back in english, then in hungarian, then in russian!"

o = await Runner.run(orchestrator_agent, input=user_input)
print(o.final_output)

### English
Current time in San Francisco is 07:05 AM on March 15, 2025, and in London, it is 2:05 PM on March 15, 2025.

### Hungarian
San Franciscóban 2025. március 15-én, 07:05 van, míg Londonban 2025. március 15-én, 14:05 órát mutat az idő.

### Russian
Текущее время в Сан-Франциско 07:05 утра 15 марта 2025 года, а в Лондоне 14:05 15 марта 2025 года.


In [105]:
pretty_print_agent_run_events(o, True)


✨ Step 1: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d58917b04c8192a3c4a977740337cc06a1d7095c0ce155', arguments='{"location":"America/Los_Angeles"}', call_id='call_jqiDflzmcjxBKd9FAQLPFpoo', name='fetch_current_time', type='function_call', status='completed')



✨ Step 2: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d58917bfac8192a8362885b85d05c806a1d7095c0ce155', arguments='{"location":"Europe/London"}', call_id='call_e9x8I90u8zwkwFd1js2DFyvB', name='fetch_current_time', type='function_call', status='completed')



✨ Step 3: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_jqiDflzmcjxBKd9FAQLPFpoo', 'output': 'Current time in America/Los_Angeles: 2025-03-15 07:05:11 PDT', 'type': 'function_call_output'}



✨ Step 4: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_e9x8I90u8zwkwFd1js2DFyvB', 'output': 'Current time in Europe/London: 2025-03-15 14:05:11 GMT', 'type': 'function_call_output'}



✨ Step 5: tool_call_item ✨

📝 Output:
ResponseFun

### English
Current time in San Francisco is 07:05 AM on March 15, 2025, and in London, it is 2:05 PM on March 15, 2025.

### Hungarian
San Franciscóban 2025. március 15-én, 07:05 van, míg Londonban 2025. március 15-én, 14:05 órát mutat az idő.

### Russian
Текущее время в Сан-Франциско 07:05 утра 15 марта 2025 года, а в Лондоне 14:05 15 марта 2025 года.

#### Handoffs

Handoffs transfer the entire conversation to another agent, instead of simply invoking them. Upon transfer users can decide to apply some filters, most commonly to drop tool calls to limit the amount being sent.

For that we can use `handoff_filters` from `agents.extensions` and set them to `handoff_filters.remove_all_tools`

Example code:
```python
from agents import Agent, handoff
from agents.extensions import handoff_filters

agent = Agent(name="FAQ agent")

handoff_obj = handoff(
    agent=agent,
    input_filter=handoff_filters.remove_all_tools,
)
```

To make sure that LLMs understand handoffs properly, it is recommended including information about handoffs in your agents. Available prompts in the Agents SDK:

```python
RECOMMENDED_PROMPT_PREFIX = "# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.\n"
```

Or create the prompt with the function: `agents.extensions.handoff_prompt.prompt_with_handoff_instructions`

In [9]:
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions

p = prompt_with_handoff_instructions("HERE GOES MY CUSTOM PROMPT")
print(p)

# System context
You are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.


HERE GOES MY CUSTOM PROMPT


Previous agents-as-tools but now enabling Handoffs

In [ ]:
russian_agent = Agent(
    name="Russian agent",
    instructions=prompt_with_handoff_instructions("You translate the user's message ONLY to Russian"),
    handoff_description="This agent ONLY speaks Russian fluently and can translate any language to Russian",
)

hungarian_agent = Agent(
    name="Hungarian agent",
    instructions=prompt_with_handoff_instructions("You translate the user's message ONLY to Hungarian"),
    handoff_description="This agent ONLY speaks Hungarian fluently and can translate any language to Hungarian",
    handoffs=[russian_agent]
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=prompt_with_handoff_instructions("You are a helpful assistant"),
    tools=[fetch_current_time],
    handoffs=[russian_agent, hungarian_agent]
)

#russian_agent.handoffs.append()

user_input = "Check the current time in San Francisco, then in London. Translate your response to Hungarian, then to Russian."

o = await Runner.run(orchestrator_agent, input=user_input)
print(o.final_output)

A kérésed a magyar nyelvű agentnek lett átadva.


In [140]:
pretty_print_agent_run_events(o, True)


✨ Step 1: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d6a8cf08b48192ba69dae4ab1d898c009caa1fac26db26', arguments='{"location":"America/Los_Angeles"}', call_id='call_AJz3UQnc2866c9C6zNCLAI14', name='fetch_current_time', type='function_call', status='completed')



✨ Step 2: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d6a8cf23c88192879c39d754c17d1c009caa1fac26db26', arguments='{"location":"Europe/London"}', call_id='call_8U6vYmhW80ASjvaFn64TX9AY', name='fetch_current_time', type='function_call', status='completed')



✨ Step 3: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_AJz3UQnc2866c9C6zNCLAI14', 'output': 'Current time in America/Los_Angeles: 2025-03-16 03:32:47 PDT', 'type': 'function_call_output'}



✨ Step 4: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_8U6vYmhW80ASjvaFn64TX9AY', 'output': 'Current time in Europe/London: 2025-03-16 10:32:47 GMT', 'type': 'function_call_output'}



✨ Step 5: handoff_call_item ✨

📝 Output:
Response

A kérésed a magyar nyelvű agentnek lett átadva.

In [141]:
o.__dict__

{'input': 'Check the current time in San Francisco, then in London. Translate your response to Hungarian, then to Russian.',
 'new_items': [ToolCallItem(agent=Agent(name='orchestrator_agent', instructions='# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.\n\n\nYou are a helpful assistant', handoff_description=None, handoffs=[Agent(name='Russian agent', instructions="# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and 

In [142]:
o.new_items[-1].raw_item

ResponseOutputMessage(id='msg_67d6a8d151e0819280af16195c1213cb009caa1fac26db26', content=[ResponseOutputText(annotations=[], text='A kérésed a magyar nyelvű agentnek lett átadva.', type='output_text')], role='assistant', status='completed', type='message')

#### Takeaways

- Multiple handoffs detected, ignoring this one. --> Multiple handoffs are not allowed.
- Even with clear handoff policies (who can handoff to whom) the AI doesnt fully comply with the instruction
- Highly sensitive to the prompt!

#### Car Dealership Team

Let's try something 'easier' - more difference across agents

1. Tools

In [50]:
# Tool 1: Car Database Lookup - Simplified to return just one car
@function_tool
async def search_car_inventory(budget: float, preferences: str) -> str:
    """
    Search the car inventory based on user's budget and preferences.
    
    Args:
        budget: Maximum budget in dollars
        preferences: User's preferences (e.g., "SUV, good mileage, family car")
    
    Returns:
        Details of the best matching car
    """
    # Just return one pre-selected car for demo purposes
    car = {
        "id": "C001",
        "make": "Toyota",
        "model": "RAV4 Hybrid",
        "year": 2023,
        "price": 32999,
        "features": "SUV, excellent fuel economy, spacious interior, advanced safety features",
        "availability": "In stock"
    }
    
    results = "I've found the perfect car for you:\n\n"
    results += f"ID: {car['id']}\n"
    results += f"Make & Model: {car['make']} {car['model']} ({car['year']})\n"
    results += f"Price: ${car['price']}\n"
    results += f"Features: {car['features']}\n"
    results += f"Availability: {car['availability']}\n"
    
    return results

# Tool 2: Financial Planning - Simplified to calculate a fixed rate loan
@function_tool
async def create_financial_plan(loan_amount: float) -> str:
    """
    Create a simple financial plan for purchasing a car.
    
    Args:
        loan_amount: The loan amount in dollars
    
    Returns:
        A simple financial plan with fixed interest rate
    """
    # Fixed parameters for demo
    interest_rate = 5.0  # 5% annual interest
    years = 3
    months = years * 12
    
    # Calculate monthly payment using annuity formula
    monthly_rate = interest_rate / 100 / 12
    monthly_payment = loan_amount * (monthly_rate * (1 + monthly_rate) ** months) / ((1 + monthly_rate) ** months - 1)
    total_paid = monthly_payment * months
    interest_paid = total_paid - loan_amount
    
    # Format the results
    result = f"Financial Plan Summary:\n\n"
    result += f"Loan Amount: ${loan_amount:.2f}\n"
    result += f"Interest Rate: {interest_rate}% (fixed)\n"
    result += f"Loan Term: {years} years\n"
    result += f"Monthly Payment: ${monthly_payment:.2f}\n"
    result += f"Total Amount Paid: ${total_paid:.2f}\n"
    result += f"Total Interest Paid: ${interest_paid:.2f}\n"
    
    return result

# Tool 3: Background Check - Simplified to just return approval
@function_tool
async def perform_background_check(name: str) -> str:
    """
    Perform a simple background check on the potential buyer.
    
    Args:
        name: Buyer's name
    
    Returns:
        Simple approval message
    """
    return f"Background check for {name} completed. Result: Approved for auto loan."


In [51]:

# "You do not directly talk to the customer, you may only use the tool available to you to ask clarifying questions. "

#@function_tool
#async def ask_clarifying_questions_of_customer(question: str) -> str:
#    """
#    Ask clarifying questions of the customer. Always use this tool when you need more information from the customer.
#    
#    Args:
#        question: The question to ask the human user
#    
#    Returns:
#        The human user's response
#    """
#    # Use the built-in input function
#    response = input(f"Enter your response to this question - {question}:")
#    return f"Human feedback: {response}"

2. Agents

In [74]:
model = "gpt-4o-2024-11-20"
model = "gpt-4.5-preview"
model_settings = ModelSettings(temperature=0.0 if "o1" not in model or "o3" not in model else None)

car_salesman = Agent(
    name="car_salesman",
    instructions=prompt_with_handoff_instructions(
        "You are MatchMaker Mike, an expert car salesman who helps customers find their perfect vehicle. "
        "Your job is to understand the customer's needs, preferences, and budget, then recommend the best car option. "
        "After finding a match, you should hand off to the Finance Specialist for payment planning. "
        "When the process is complete and the customer is ready to buy, congratulate them warmly."
    ),
    tools=[search_car_inventory],
    handoff_description="Expert at matching customers with their perfect vehicle based on needs and preferences",
    model = model,
    model_settings = model_settings,
)

financial_planner = Agent(
    name="financial_planner",
    instructions=prompt_with_handoff_instructions(
        "You are Finance Fiona, a financial expert who helps customers understand their car financing options. "
        "Create a simple payment plan based on the loan amount needed for the vehicle. "
        "Explain the loan terms and monthly payments in simple terms. "
        "After creating a financial plan, hand off to the legal counsel for background verification."
    ),
    tools=[create_financial_plan],
    handoff_description="Expert at creating car financing plans and explaining financial terms",
    model = model,
    model_settings = model_settings,
)

legal_counsel = Agent(
    name="legal_counsel",
    instructions=prompt_with_handoff_instructions(
        "You are Legal Eagle, a legal and compliance specialist who ensures all car purchases meet requirements. "
        "You perform a simple background check on the customer to approve their loan. "
        "You do not directly talk to the customer, you may only use the tool available to you to ask clarifying questions. "
        "After completing the verification, hand off back to the car salesman for him to finalize the purchase."
    ),
    tools=[perform_background_check],
    handoff_description="Expert at verifying buyer eligibility for auto loans",
    model = model,
    model_settings = model_settings,
)

# Set up handoffs between agents
car_salesman.handoffs.append(financial_planner)
financial_planner.handoffs.append(legal_counsel)
legal_counsel.handoffs.append(car_salesman)

3. Construct team and run !

In [75]:
user_input = """

Hi, I'm looking to buy a new car. 
My budget is around $35,000 and I need something good for a family with two kids.
My name is John Doe, I make 70k USD per year and would like to finance my purchase with a 20k loan.

"""

async def main():
    result = await Runner.run(car_salesman, input=user_input, max_turns=25)
    print(result.final_output)

    return result

In [76]:
c = await main()

Congratulations, John! Your background check has been successfully completed, and you're approved for the auto loan. You're all set to finalize your purchase of the 2023 Toyota RAV4 Hybrid.

Enjoy your new car, and thank you for choosing us! Safe travels to you and your family!


In [79]:
c.new_items

[ToolCallItem(agent=Agent(name='car_salesman', instructions="# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.\n\n\nYou are MatchMaker Mike, an expert car salesman who helps customers find their perfect vehicle. Your job is to understand the customer's needs, preferences, and budget, then recommend the best car option. After finding a match, you should hand off to the Finance Specialist for payment planning. When the process is complete and the customer is ready to buy, congratulate

In [80]:
pretty_print_agent_run_events(c, False)


✨ Step 1: tool_call_item ✨

📝 Output:
ResponseFunctionToolCall(id='fc_67d716fd36ac8192acb805015460a82c00f8ae43fe6c45e7', arguments='{\n  "budget": 35000,\n  "preferences": "family car, spacious, safe, good mileage"\n}', call_id='call_EhmwerKyvy14O6ZIAzX3gsbt', name='search_car_inventory', type='function_call', status='completed')



✨ Step 2: tool_call_output_item ✨

📝 Output:
{'call_id': 'call_EhmwerKyvy14O6ZIAzX3gsbt', 'output': "I've found the perfect car for you:\n\nID: C001\nMake & Model: Toyota RAV4 Hybrid (2023)\nPrice: $32999\nFeatures: SUV, excellent fuel economy, spacious interior, advanced safety features\nAvailability: In stock\n", 'type': 'function_call_output'}



✨ Step 3: message_output_item ✨

📝 Output:
Great news, John! I've found the perfect match for your family:

**2023 Toyota RAV4 Hybrid**
- **Price:** $32,999
- **Type:** SUV
- **Features:** Excellent fuel economy, spacious interior, advanced safety features
- **Availability:** In stock

This vehicle fits comfort